<a href="https://colab.research.google.com/github/snpsuen/Deep_Learning_Data/blob/main/script/finetune_gpt2_example03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install transformers datasets

In [ ]:
import transformers
print(transformers.__version__)

4.53.3


In [ ]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset('imdb')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
from transformers import GPT2Tokenizer

# Load the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Set the padding token
tokenizer.pad_token = tokenizer.eos_token

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=512)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
from transformers import GPT2LMHeadModel

# Load the model
model = GPT2LMHeadModel.from_pretrained('gpt2')

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    max_steps=20,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
)

In [ ]:
from transformers import Trainer, DataCollatorForLanguageModeling

# Data collator for language modeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    data_collator=data_collator,
)

In [ ]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=20, training_loss=3.825300598144531, metrics={'train_runtime': 55.3839, 'train_samples_per_second': 0.722, 'train_steps_per_second': 0.361, 'total_flos': 10451681280000.0, 'train_loss': 3.825300598144531, 'epoch': 0.0016})

In [ ]:
import torch
from torch.utils.data import DataLoader
import math
from tqdm.auto import tqdm

def custom_eval(model, tokenized_datasets, tokenizer, data_collator, training_args, num_steps):
    """
    Performs a custom evaluation for a limited number of steps.

    Args:
        model: The model to evaluate.
        tokenized_datasets: The tokenized dataset dictionary.
        tokenizer: The tokenizer used for the model.
        data_collator: The data collator for preparing batches.
        training_args: The training arguments containing batch size information.
        num_steps: The number of evaluation steps to perform.

    Returns:
        A dictionary containing the evaluation results (average loss and perplexity).
    """
    # Create a smaller evaluation dataset
    # Ensure we don't request more samples than available in the dataset
    max_samples = len(tokenized_datasets['test'])
    num_samples_for_eval = min(num_steps * training_args.per_device_eval_batch_size, max_samples)
    small_eval_dataset = tokenized_datasets['test'].select(range(num_samples_for_eval))

    # Remove the 'text' column as it's not needed for evaluation after tokenization
    small_eval_dataset = small_eval_dataset.remove_columns(['text'])

    # Create a DataLoader for the smaller evaluation dataset.
    eval_dataloader = DataLoader(
        small_eval_dataset,
        batch_size=training_args.per_device_eval_batch_size,
        collate_fn=data_collator
    )

    # Set the model to evaluation mode.
    model.eval()

    # Initialize variables to accumulate loss and track the number of steps.
    total_eval_loss = 0
    num_eval_steps = 0

    # Get the device from the model
    device = model.device

    print(f"Performing custom evaluation for {num_steps} steps...")

    # Iterate over a limited number of batches from the DataLoader.
    # Use tqdm for a progress bar
    for step, batch in enumerate(tqdm(eval_dataloader, total=num_steps)):
        if step >= num_steps:
            break

        # Move batch to the appropriate device (GPU/CPU).
        batch = {k: v.to(device) for k, v in batch.items()}

        # Perform a forward pass through the model to get the loss.
        with torch.no_grad():
            # Ensure 'labels' are present for loss calculation
            if 'labels' not in batch:
                 batch['labels'] = batch['input_ids']

            outputs = model(**batch)
            loss = outputs.loss

        # Accumulate the loss and increment the step count.
        total_eval_loss += loss.item()
        num_eval_steps += 1

    # After the loop, calculate the average loss and perplexity.
    results = {}
    if num_eval_steps > 0:
        average_eval_loss = total_eval_loss / num_eval_steps
        perplexity = math.exp(average_eval_loss)

        results['Average Loss'] = average_eval_loss
        results['Perplexity'] = perplexity

        print(f"Custom Evaluation - Average Loss: {average_eval_loss:.4f}")
        print(f"Custom Evaluation - Perplexity: {perplexity:.4f}")
    else:
        print("No evaluation steps were performed.")
        results['Average Loss'] = None
        results['Perplexity'] = None

    return results

In [ ]:
# Define the number of steps for the custom evaluation
num_steps_for_custom_eval = 20

# Call the custom evaluation function
results = custom_eval(
    model=model,
    tokenized_datasets=tokenized_datasets,
    tokenizer=tokenizer,
    data_collator=data_collator,
    training_args=training_args,
    num_steps=num_steps_for_custom_eval
)

# Print the perplexity from the results
if results and 'Perplexity' in results:
    print(f"Perplexity from custom evaluation: {results['Perplexity']:.4f}")
else:
    print("Custom evaluation did not return perplexity.")

Performing custom evaluation for 20 steps...


  0%|          | 0/20 [00:00<?, ?it/s]

Custom Evaluation - Average Loss: 3.6442
Custom Evaluation - Perplexity: 38.2523
Perplexity from custom evaluation: 38.2523


In [ ]:
model.save_pretrained('./fine_tuned_gpt2')
tokenizer.save_pretrained('./fine_tuned_gpt2')

('./fine_tuned_gpt2/tokenizer_config.json',
 './fine_tuned_gpt2/special_tokens_map.json',
 './fine_tuned_gpt2/vocab.json',
 './fine_tuned_gpt2/merges.txt',
 './fine_tuned_gpt2/added_tokens.json')

In [ ]:
from transformers import pipeline

# Load the fine-tuned model and tokenizer
model = GPT2LMHeadModel.from_pretrained('./fine_tuned_gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('./fine_tuned_gpt2')

# Create a text generation pipeline
text_generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

# Generate text
prompt = "Once upon a time"
generated_text = text_generator(prompt, max_length=100, num_return_sequences=1)
print(generated_text[0]['generated_text'])

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Once upon a time of war, the nation of India had a great deal to do with the war which had been waged by the British. But the nation of India, and not the British, was the enemy. The war of independence was the only victory of all the British powers in the war against Indus. It was the first victory of any Indian in the history of the world.

But what was the motive behind this victory? "The people of India wanted to maintain a monarchy," said a young man. He was a fellow of India's Congress party and the heir of a great colony. He was one of the chief leaders of the national Congress, and he was a member of the government of India in the early years of the war.

The Indians of India, then, had a great feeling of good reason to support the King of India. They were well aware that the British had been the enemy and had been waging war against Indus. The British wanted to maintain the monarchy for the sake of its own political and economic power. The Indian people had no such feeling. In

In [ ]:
# --- Interactive Prompt ---
print("\n🎭 FTgpt2 Interactive Mode (word-level) — type 'exit' to quit.")
while True:
    prompt = input("\nYou > ").strip()
    if prompt.lower() in ['exit', 'quit']:
        print("Goodbye!")
        break
    if not prompt:
        continue
    try:
        generated_text = text_generator(prompt, max_length=100, num_return_sequences=1)
        print("\nFTgpt2 >", generated_text[0]['generated_text'])
    except Exception as e:
        print(f"⚠️ Error: {e}")
        continue


🎭 FTgpt2 Interactive Mode (word-level) — type 'exit' to quit.

You > A long long time ago


Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



FTgpt2 > A long long time ago, when the world was still reeling from the terrible floods that swept across the world, the world was not so fortunate. The flood of water was so great that there were only a handful of people left alive to save the world. The rest of the world just went on and on as they were supposed to do. And just as the world collapsed, it was told that the flood would end soon. And the world went on and on, until the world collapsed and there was nothing left to do but get on with it. And then, just as the world collapsed, the world went on, until nothing left to do was left to do and the world went on, until nothing existed to help. And then, just as the world collapsed, the world went on, until nothing existed to help. And then, just as the world collapsed, the world went on, until nothing existed to help. And then, just as the world collapsed, the world went on, until nothing existed to help. And then, just as the world collapsed, the world went on, until nothing

Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



FTgpt2 > Are you ready?

We are going to be on a mission to make the most of your $99.99. But first, let's start with some basics:

How to Buy a New PC?

To begin, you'll need to register for the Steam service and log in to your account to gain access to the online store. Once logged in, enter your Steam ID and click "Purchase" to purchase your new PC.

Once you've purchased your new PC, click the "Buy Now" button.

Once you've purchased your new PC, click the "Wait For More Information" button.

Once you've purchased your new PC, click the "Buy Now" button.

Once you've purchased your new PC, click the "Buy Now" button.

Once you've purchased your new PC, click the "Wait For More Information" button.

Once you've purchased your new PC, click the "Buy Now" button.

Once you've purchased your new PC, click the "Wait For More Information" button.

Once you've purchased your new PC, click the "Wait For More Information" button.

Once you've purchased your new PC, click the "Wait For More

Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



FTgpt2 > The good, the bad and the ugly - the more of them you become.

And if you want to take the hard stuff and take all the hard stuff, then just go see a movie.

I never said I'm going to be the hero. But I think some people just feel I can't do that and that's good.

And I like seeing a lot of great actors.

It's always fun to see a lot of great actors.

And I like seeing actors who are so good.

They're so good because they're so special.

They're so good because they're so much bigger than anything that we've ever seen.

Sometimes you just can't see things that you can't see.

And they're so great because they're so, so special.

And I love seeing actors who are so good.

I love seeing actors who are so special.

And I love seeing actors who are so, so special.

The only reason I love going to a theater is to see great actors.

I love watching great actors.

And I love seeing great actors who are so good.

And I love watching great actors who are so, so special.

You > exit
Go